In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
ticker_list = stock.get_market_ticker_list(market="KOSDAQ")

#ticker_list=[]
# 빈 리스트
results_list = []

# 각 종목에 대해 반복
for ticker in tqdm(ticker_list, desc="Processing"):
    try:
        ticker_name = stock.get_market_ticker_name(ticker)
        start_date = "2020-03-20"
        end_date = "2024-03-20"
        # 주어진 기간 동안의 일별 거래량 정보 가져오기
        df = stock.get_market_ohlcv_by_date(fromdate=start_date, todate=end_date, ticker=ticker)


        def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
            """MACD 및 MACD 신호 계산"""
            df['EMA_short'] = df['종가'].ewm(span=short_window, adjust=False).mean()
            df['EMA_long'] = df['종가'].ewm(span=long_window, adjust=False).mean()
            df['MACD'] = df['EMA_short'] - df['EMA_long']
            df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
            return df

        def calculate_rsi(df, window=14, signal_window=9):
            """RSI 및 RSI 신호 계산"""
            delta = df['종가'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

            rs = gain / loss
            df['RSI'] = 100 - (100 / (1 + rs))
            
            # RSI 신호선 추가
            df['RSI_Signal'] = df['RSI'].ewm(span=signal_window, adjust=False).mean()
            return df

        # MACD 및 MACD 신호 계산
        df = calculate_macd(df)

        df = calculate_rsi(df)


        # 지정된 기간에 대한 SMA 계산
        periods = [5, 20, 60, 120, 240]
        for period in periods:
            df[f'SMA_{period}'] = df['종가'].rolling(window=period).mean()

        # 지정된 기간에 대한 EMA 계산
        for period in periods:
            df[f'EMA_{period}'] = df['종가'].ewm(span=period, adjust=False).mean()

        for i in range(len(periods)):
            for j in range(i + 1, len(periods)):
                smaller_period = periods[i]
                larger_period = periods[j]
                df[f'SMA_{smaller_period}_minus_SMA_{larger_period}'] = df[f'SMA_{smaller_period}'] - df[f'SMA_{larger_period}']

        # EMA 간의 차이 계산
        for i in range(len(periods)):
            for j in range(i + 1, len(periods)):
                smaller_period = periods[i]
                larger_period = periods[j]
                df[f'EMA_{smaller_period}_minus_EMA_{larger_period}'] = df[f'EMA_{smaller_period}'] - df[f'EMA_{larger_period}']

        for column in ['시가', '고가', '저가','거래량','MACD','MACD_Signal','RSI','RSI_Signal']:
            df[f'{column}_등락률'] = df[column].pct_change() * 100

        # SMA 및 EMA의 전 거래일 대비 등락률 계산 및 DataFrame에 추가
        periods = [5, 20, 60, 120, 240]

        # SMA 등락률 계산 및 추가
        for period in periods:
            df[f'SMA_{period}_등락률'] = df[f'SMA_{period}'].pct_change() * 100

        # EMA 등락률 계산 및 추가
        for period in periods:
            df[f'EMA_{period}_등락률'] = df[f'EMA_{period}'].pct_change() * 100

        # 'MACD'가 0 이상일 때 1, 아니면 0을 할당
        df['MACD_Binary'] = (df['MACD'] >= 0).astype(int)

        # 'MACD - MACD_Signal'이 0 이상일 때 1, 아니면 0을 할당
        df['MACD_minus_Signal_Binary'] = ((df['MACD'] - df['MACD_Signal']) >= 0).astype(int)

        # 'RSI - RSI_Signal'이 0 이상일 때 1, 아니면 0을 할당
        df['RSI_minus_Signal_Binary'] = ((df['RSI'] - df['RSI_Signal']) >= 0).astype(int)

        # 시가 대비 종가 등락률 컬럼 추가
        df['시가_대비_종가_등락률'] = ((df['종가'] - df['시가']) / df['시가']) * 100
        df['시가_대비_저가_등락률'] = ((df['저가'] - df['시가']) / df['시가']) * 100
        df['시가_대비_고가_등락률'] = ((df['고가'] - df['시가']) / df['시가']) * 100
        df['저가_대비_종가_등락률'] = ((df['종가'] - df['저가']) / df['저가']) * 100
        df['저가_대비_고가_등락률'] = ((df['고가'] - df['저가']) / df['저가']) * 100
        df['고가_대비_종가_등락률'] = ((df['종가'] - df['고가']) / df['고가']) * 100
        # 종가 - SMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
        for period in [5, 20, 60, 120, 240]:
            df[f'종가_minus_SMA_{period}'] = df['종가'] - df[f'SMA_{period}']

        # 종가 - EMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
        for period in [5, 20, 60, 120, 240]:
            df[f'종가_minus_EMA_{period}'] = df['종가'] - df[f'EMA_{period}']


        # 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
        df['next_day_return'] = df['등락률'].shift(-1)
        df['target'] = df['next_day_return'].apply(lambda x: 1 if x > 0.25 else -1)
        df.dropna(inplace=True)  # 마지막 행 삭제

        first_column_name = df.columns[0]

        
        X = df.drop(['next_day_return','target'], axis=1)
        y = df['target']

        # Calculating correlations for Forward Selection
        correlations = X.corrwith(y).abs().sort_values(ascending=False).reset_index()
        correlations.columns = ['Feature', 'Correlation']

        top_8_features = correlations.sort_values(by='Correlation', ascending=False).head(8)['Feature']

        # X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)
        X_top8 = X[top_8_features]
        y = df['target'] # y 값을 0과 1로 조정

        # 데이터를 학습 세트와 테스트 세트로 분할
        X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=42)

        # 선형회귀 모델 학습 및 평가
        linear_reg = LinearRegression()
        linear_reg.fit(X_train, y_train)
        y_pred_linear = linear_reg.predict(X_test)
        r2_linear = r2_score(y_test, y_pred_linear)

        # 다항회귀 모델 학습 및 평가
        poly_features = PolynomialFeatures(degree=2, include_bias=False)
        X_poly_train = poly_features.fit_transform(X_train)
        X_poly_test = poly_features.transform(X_test)
        poly_reg = LinearRegression()
        poly_reg.fit(X_poly_train, y_train)
        y_pred_poly = poly_reg.predict(X_poly_test)
        r2_poly = r2_score(y_test, y_pred_poly)

        # 라쏘 회귀 모델 학습 및 평가
        lasso_reg = Lasso(alpha=0.1)
        lasso_reg.fit(X_train, y_train)
        y_pred_lasso = lasso_reg.predict(X_test)
        r2_lasso = r2_score(y_test, y_pred_lasso)

        # 릿지 회귀 모델 학습 및 평가
        ridge_reg = Ridge(alpha=1.0)
        ridge_reg.fit(X_train, y_train)
        y_pred_ridge = ridge_reg.predict(X_test)
        r2_ridge = r2_score(y_test, y_pred_ridge)

        # 결과 저장
        results_list.append({
            'Ticker': ticker,
            'Name': ticker_name,
            'r2_linear': r2_linear,
            'r2_poly': r2_poly,
            'r2_lasso': r2_lasso,
            'r2_ridge': r2_ridge
        })
        pass
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        continue

# 결과를 데이터프레임으로 변환하고 precision_svm 값에 따라 내림차순 정렬
results_df = pd.DataFrame(results_list).sort_values(by='r2_linear', ascending=False)



Processing:   0%|          | 3/1721 [00:04<33:13,  1.16s/it]  c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.478e+02, tolerance: 5.554e-02
  model = cd_fast.enet_coordinate_descent(
Processing:   0%|          | 4/1721 [00:05<23:09,  1.24it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.277e+01, tolerance: 5.181e-02
  model = cd_fast.enet_coordinate_descent(
Processing:   0%|          | 8/1721 [00:06<09:22,  3.04it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: C

Error processing 456440: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


Processing:   1%|          | 20/1721 [00:08<05:50,  4.86it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.509e-01, tolerance: 5.543e-02
  model = cd_fast.enet_coordinate_descent(
Processing:   1%|          | 21/1721 [00:09<07:41,  3.69it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.601e+01, tolerance: 5.400e-02
  model = cd_fast.enet_coordinate_descent(
Processing:   1%|▏         | 23/1721 [00:11<21:04,  1.34it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: 

Error processing 440290: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


Processing:   2%|▏         | 34/1721 [00:17<13:37,  2.06it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.358e+02, tolerance: 5.422e-02
  model = cd_fast.enet_coordinate_descent(
Processing:   2%|▏         | 36/1721 [00:18<12:12,  2.30it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.636e+02, tolerance: 5.377e-02
  model = cd_fast.enet_coordinate_descent(
Processing:   2%|▏         | 40/1721 [00:19<09:12,  3.04it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: 

...

c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.841e+02, tolerance: 5.563e-02
  model = cd_fast.enet_coordinate_descent(
Processing:  97%|█████████▋| 1665/1721 [08:22<00:15,  3.72it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.156e+01, tolerance: 5.422e-02
  model = cd_fast.enet_coordinate_descent(
Processing:  97%|█████████▋| 1666/1721 [08:23<00:14,  3.85it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might

Error processing 455310: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.693e+01, tolerance: 5.310e-02
  model = cd_fast.enet_coordinate_descent(
Processing:  98%|█████████▊| 1678/1721 [08:26<00:12,  3.56it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.157e+02, tolerance: 5.473e-02
  model = cd_fast.enet_coordinate_descent(
Processing:  98%|█████████▊| 1682/1721 [08:27<00:12,  3.23it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might

Error processing 460930: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.315e+01, tolerance: 5.473e-02
  model = cd_fast.enet_coordinate_descent(
Processing:  98%|█████████▊| 1692/1721 [08:29<00:06,  4.22it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.862e+01, tolerance: 1.929e-02
  model = cd_fast.enet_coordinate_descent(
Processing:  99%|█████████▊| 1696/1721 [08:30<00:05,  4.20it/s]c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might

In [2]:
import pandas as pd

# 전체 행을 출력하도록 설정
pd.set_option('display.max_rows', None)

# 전체 열을 출력하도록 설정
pd.set_option('display.max_columns', None)

results_df

,Ticker,Name,r2_linear,r2_poly,r2_lasso,r2_ridge
1562,127980,화인써키트,0.718620,-1.111521e+02,0.484190,0.631083
872,439410,엔에이치스팩26호,0.312206,-1.309937e+01,0.144933,0.168963
1478,430230,하나금융24호스팩,0.310635,-2.038798e+00,0.193229,0.228701
479,445360,비엔케이제1호스팩,0.306890,-4.434131e+01,0.126803,0.224122
877,208860,엔지스테크널러지,0.252425,-8.624591e-01,-0.020803,0.225990
643,419270,신영스팩7호,0.218950,-1.482724e+00,0.158811,0.218924
1381,425040,티이엠씨,0.210676,-7.794147e+02,0.167552,0.042796
41,439730,IBKS제20호스팩,0.170439,-1.266237e+02,0.082981,0.158134
1047,430700,유안타제9호스팩,0.162301,-3.970115e-01,0.092005,0.160450
869,422040,엔에이치스팩23호,0.149179,-7.333962e-01,0.100582,0.174960
